In [3]:
import numpy as np
from pybaseball import statcast
data = statcast(start_dt='2018-04-07', end_dt='2018-11-02')


This is a large query, it may take a moment to complete
Completed sub-query from 2018-04-07 to 2018-04-12
Completed sub-query from 2018-04-13 to 2018-04-18
Completed sub-query from 2018-04-19 to 2018-04-24
Completed sub-query from 2018-04-25 to 2018-04-30
Completed sub-query from 2018-05-01 to 2018-05-06
Completed sub-query from 2018-05-07 to 2018-05-12
Completed sub-query from 2018-05-13 to 2018-05-18
Completed sub-query from 2018-05-19 to 2018-05-24
Completed sub-query from 2018-05-25 to 2018-05-30
Completed sub-query from 2018-05-31 to 2018-06-05
Completed sub-query from 2018-06-06 to 2018-06-11
Completed sub-query from 2018-06-12 to 2018-06-17
Completed sub-query from 2018-06-18 to 2018-06-23
Completed sub-query from 2018-06-24 to 2018-06-29
Completed sub-query from 2018-06-30 to 2018-07-05
Completed sub-query from 2018-07-06 to 2018-07-11
Completed sub-query from 2018-07-12 to 2018-07-17
Completed sub-query from 2018-07-18 to 2018-07-23
Completed sub-query from 2018-07-24 to 2018-

In [4]:
df = data[[ 'at_bat_number',
    'events',
 'batter',
 'on_3b',
 'on_2b',
 'on_1b',
 'home_score',
'away_score'
]]
df = df.iloc[::-1]
df["Afterhit_1b"] = df['on_1b'].shift(-1)
df["Afterhit_2b"] = df['on_2b'].shift(-1)
df["Afterhit_3b"] = df['on_3b'].shift(-1)
df["Pre_bat_runs"] = df['home_score']+df['away_score']
df["Post_bat_runs"] = df['Pre_bat_runs'].shift(-1)
df = df.dropna(subset=['events'])
df = df.fillna(0)

#df = df.loc[(df['at_bat_number'] = 30)]
df.head()

,at_bat_number,events,batter,on_3b,on_2b,on_1b,home_score,away_score,Afterhit_1b,Afterhit_2b,Afterhit_3b,Pre_bat_runs,Post_bat_runs
698627,1.0,walk,444482.0,0.0,0.0,0.0,0.0,0.0,444482.0,0.0,0.0,0.0,0.0
698624,2.0,field_out,606466.0,0.0,0.0,444482.0,0.0,0.0,444482.0,0.0,0.0,0.0,0.0
698619,3.0,walk,502671.0,0.0,0.0,444482.0,0.0,0.0,502671.0,444482.0,0.0,0.0,0.0
698617,4.0,single,572041.0,0.0,444482.0,502671.0,0.0,0.0,572041.0,0.0,502671.0,0.0,1.0
698610,5.0,walk,518614.0,502671.0,0.0,572041.0,0.0,1.0,518614.0,572041.0,502671.0,1.0,1.0


In [6]:
batters = {}

def main():
    for index, row in df.iterrows():
        #if there was a change off the bases
        #if there are no other people on base
        if((row['on_2b']  == 0) & (row['on_3b'] == 0) & (row['on_1b'] == 0)):
            #home run case
            if(row['events'] == 'home_run'):
                if row['batter'] not in batters.keys():
                    batters[row['batter']] = 4
                else:
                    batters[row['batter']] =  batters[row['batter']] + 4
            #triple Case
            elif(row['Afterhit_3b'] >= 0):
                if row['batter'] not in batters.keys():
                    batters[row['batter']] = 3
                else:
                    batters[row['batter']] =  batters[row['batter']] + 3
            #double case
            elif (row['Afterhit_2b'] >= 0):
                if row['batter'] not in batters.keys():
                    batters[row['batter']] = 2
                else:
                    batters[row['batter']] =  batters[row['batter']] + 2
            #gets on base in various ways 
            elif(row['Afterhit_1b'] >= 0):
                if row['batter'] not in batters.keys():
                    batters[row['batter']] = 1
                else:
                    batters[row['batter']] =  batters[row['batter']] + 1
        #there are people on base
        else:
            #This at bat did not change the score
            if(row['Pre_bat_runs'] == row['Post_bat_runs']  ):
                #first made it to second
                if(row['on_1b'] == row['Afterhit_2b'] ):
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 1  
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 1
                #second made it to third
                if(row['on_2b'] == row['Afterhit_3b'] ):
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 1  
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 1
                #if batter made it to first
                if(row['batter'] == row['Afterhit_1b'] ):
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 1  
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 1
                #if batter made it to second 
                if(row['batter'] == row['Afterhit_2b'] ):
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 2  
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 2
                #if 1st made it to third 
                if(row['on_1b'] == row['Afterhit_3b'] ):
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 2  
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 2
            #if one run was scored
            elif(row['Pre_bat_runs']+1 == row['Post_bat_runs']  ):
                #case where most likely the person On third bases scored. Do to the data I won't know who for sure scored the run I have to make and assumtion
                if((row['on_3b'] > 0)):
                   #accounts for the runner person on third scoring
                    if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1 
                    else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #second made it to third
                    if(row['on_2b'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #first made it  to second
                    if(row['on_1b'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #first makes it to third
                    if(row['on_1b'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
                #This is assuming the person on second scores again I don't know for sure 
                elif((row['on_3b'] == 0) & (row['on_2b'] >= 0) ):
                   #accounts for the runner person on second scoring
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 2 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 2
                    #first made it  to second
                    if(row['on_1b'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #first makes it to third
                    if(row['on_1b'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
                # the person on 1st ends up scoring
                else:
                   #accounts for the runner person on first scoring
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 3 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 3
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
            #two runs scored
            elif(row['Pre_bat_runs']+2 == row['Post_bat_runs']):
                #if third and second scored. 
                if((row['on_3b'] >= 0) & (row['on_2b'] >= 0) & (row['on_1b'] == 0) ):
                   #accounts for the runners scoring 
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 3 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 3                     
                    #first made it  to second
                    if(row['on_1b'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #first makes it to third
                    if(row['on_1b'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2 
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3                     
                elif((row['on_3b'] >= 0) & (row['on_1b'] >= 0) &(row['on_2b'] == 0)):
                    #assuming first and third scored this assumes that there was no one on second
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 4 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 4     
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
                    
                elif((row['on_3b'] >= 0) & (row['on_1b'] == 0) &(row['on_2b'] == 0)):
                     #assuming batter and third scored this assumes that there was no one on second
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 5 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 5
                elif((row['on_3b'] == 0) & (row['on_1b'] >= 0) &(row['on_2b'] >= 0)):
                     #assuming first and second scored this assumes that there was no one on third
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 5 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 5
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
                elif((row['on_3b'] == 0) & (row['on_1b'] == 0) &(row['on_2b'] >= 0)):
                     #assuming batter and second scored this assumes that there was no one on second
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 6 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 6
                
                elif((row['on_3b'] == 0) & (row['on_1b'] >= 0) &(row['on_2b'] == 0)):
                     #assuming batter and first scored this assumes that there was no one on second and third
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 7 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 7
            
            elif(row['Pre_bat_runs']+3 == row['Post_bat_runs']):
                #if bases are loaded and all runners score
                if((row['on_3b'] >= 0) & (row['on_1b'] >= 0) &(row['on_2b'] >= 0)):
                     #assuming batter and first scored this assumes that there was no one on second
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 7 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 7
                    #if batter made it to first
                    if(row['batter'] == row['Afterhit_1b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 1  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 1
                    #if batter made it to second 
                    if(row['batter'] == row['Afterhit_2b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 2  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 2
                    #if batter made it to third 
                    if(row['batter'] == row['Afterhit_3b'] ):
                        if row['batter'] not in batters.keys():
                            batters[row['batter']] = 3  
                        else:
                            batters[row['batter']] =  batters[row['batter']] + 3
                elif((row['on_3b'] >= 0) & (row['on_1b'] == 0) &(row['on_2b'] >= 0)):
                     #assuming 3rd batter and second score
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 7 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 7
                elif((row['on_3b'] >= 0) & (row['on_1b'] >= 0) &(row['on_2b'] == 0)):
                     #assuming batter and first scored  and third
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 8 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 8
                elif((row['on_3b'] == 0) & (row['on_1b'] >= 0) &(row['on_2b'] >= 0)):
                    #assuming 2,b,1 score
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 9 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 9
            else:
                #grand slam conditon
                if((row['on_3b'] >= 0) & (row['on_1b'] >= 0) &(row['on_2b'] >= 0)):
                     #assuming batter and first scored this assumes that there was no one on second
                    if row['batter'] not in batters.keys():
                        batters[row['batter']] = 10 
                    else:
                        batters[row['batter']] =  batters[row['batter']] + 10
                
main()
print(batters)


{444482.0: 1634, 606466.0: 1504, 502671.0: 1775, 572041.0: 1141, 518614.0: 1105, 451594.0: 833, 502054.0: 1467, 572761.0: 1788, 542303.0: 1617, 500874.0: 1578, 425877.0: 1312, 425772.0: 542, 425844.0: 189, 657557.0: 1258, 543939.0: 1038, 608379.0: 74, 572008.0: 764, 502481.0: 594, 664056.0: 1160, 605113.0: 1521, 488671.0: 598, 542255.0: 1744, 645277.0: 1885, 518692.0: 1938, 455976.0: 1867, 453568.0: 1941, 518934.0: 1615, 571448.0: 1827, 471865.0: 1296, 605512.0: 434, 621020.0: 1355, 475247.0: 402, 435622.0: 1681, 467827.0: 1192, 596115.0: 1736, 547172.0: 597, 542208.0: 172, 434671.0: 95, 518452.0: 93, 572669.0: 66, 642162.0: 346, 435559.0: 1043, 455104.0: 909, 641820.0: 1602, 592518.0: 1970, 570731.0: 1295, 458731.0: 1621, 592450.0: 1352, 519317.0: 1833, 592122.0: 656, 430945.0: 1535, 448801.0: 1296, 542921.0: 1023, 609280.0: 1637, 607054.0: 695, 519222.0: 706, 591720.0: 238, 476883.0: 317, 623993.0: 211, 642082.0: 474, 544369.0: 1520, 664023.0: 1181, 592178.0: 1143, 656941.0: 1253, 57

987
987
